## Installation

In [ ]:
pip install azure-ai-vision

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install openai==0.28.1

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import azure.ai.vision as cvsdk
from PIL import Image, ImageDraw
import numpy as np

import openai
import json
import pandas as pd

In [ ]:
#Setting up connection to Vision Studio

endpoint = 'https://visionstudioinstance.cognitiveservices.azure.com/'
key = '<YOUR API KEY>'

# Create a Vision Service
service_options = cvsdk.VisionServiceOptions(endpoint, key)

In [ ]:
# Select an image to analyze
img_filename = "receipt.jpg"
vision_source = cvsdk.VisionSource(filename=img_filename)


# Set image analysis options and features
# cvsdk.ImageAnalysisFeature.TEXT will specify that we are using text extraction service
analysis_options = cvsdk.ImageAnalysisOptions()
analysis_options.features = (
    cvsdk.ImageAnalysisFeature.TEXT
)


# Get the text extracted from image
image_analyzer = cvsdk.ImageAnalyzer(service_options, vision_source, analysis_options)
result = image_analyzer.analyze()

# Creating an empty list to append the extracted text line by line
extracted_lines = []

# Checking if the extraction was successful
if result.reason == cvsdk.ImageAnalysisResultReason.ANALYZED:
    # Checking if there is any text in the result
    if result.text is not None:
        # Reading all lines of the text
        for line in result.text.lines:
            extracted_lines.append(line.content)
        # Joining all the lines to create a large string
        final_extracted_lines = '\n'.join(extracted_lines)

else:
    # If there was an error, print the information
    error_details = cvsdk.ImageAnalysisErrorDetails.from_result(result)
    print("Analysis failed.")
    print(f" Error reason: {error_details.reason}")
    print(f" Error code: {error_details.error_code}")
    print(f" Error message: {error_details.message}")

In [ ]:
final_extracted_lines

"COSTCO\nWHOLESALE\nKanata #541\n770 Silver Seven Rd\nKanata, ON K2V 0A1\n1S Member 111864865659\n307987 CHARCOAL 8KG\n17.99\nH\n1236258 BAMBOO DRAWR\n82.99\nH\n4696334 RUFFLES ALL\n5.99\nH\n1561688 CK BRA 2PK\n19.99\n1632921 DENSIFY\n19.99\nH\n1669990 TPD/1632921\n4.00- H\n1400615 ORG BEANS\n328795 RIVAJ NAAN\n1.99\n1596 MUSHROOMS\n4.39\n1673656 TPD/1596\n5.49\n38429 YOGOURT 2X1K\n1.50-\n1523795 ORGANIC DATE\n5.99\n1202649 LRG FREE RUN\n7.99\n1161625 SPAGHETTI 4K\n7.89\n1170871 SHAKE'N FEED\n9.99\n1670106 TPD/1170871\n19.99\nH\n21928 HALAL GROUND\n4.00- 1\n11348 B/S THIGHS\n29.99\nSUBTOTAL\n28.51\nTAX\n213.67\n**** TOTAL\n14.16\n227.83\nXXXXXXXXXXXX3967\nACCT: MASTERCARD\nREFERENCE #: 66224273-0010013480 C\nAUTH #: 6030E\n2022/06/03 20:02:00\nInvoice Number: 201348\nPurchase - CAPITAL ONE\nA0000000041010\n0000008000 E800\n01 APPROVED - THANK YOU 027\nAMOUNT: 227.83\nIMPORTANT - retain this copy\nfor your records\nCUSTOMER COPY\nMasterCard\n227.83\nCHANGE\n0.00\n(H) HST 13%\n14.16\nTOT

## Authentication
Find the information below from Microsoft Azure

In [ ]:
# Setting up Openai connections

openai.api_key = "<your_azure_openai_key>"
openai.api_base = "<your_azure_openai_base>"
openai.api_type = "<your_azure_openai_type>"
openai.api_version = "2023-07-01-preview"
chat_model_id = "<your model id>"


In [ ]:
system_message = """You are a helpful bot assistant,
your task is read the text of reciept and make a html table.
This table has columns named store name, date, item name, item category, price.
Please identify what items were purchased and categorize them into one of these categories:
1. Food items
2. Transportation
3. Home improvements and cunstructions.
4. Beauty and healthcare
write the html code for this table.
Here is the text extracted from the receipt:"""
query = final_extracted_lines

In [ ]:
prompt = [{'role': 'system','content': system_message},
          {'role': 'user','content': query}]

In [ ]:
response = openai.ChatCompletion.create(
    deployment_id=chat_model_id,
    messages=prompt,
    temperature=0
)

In [ ]:
from IPython.core.display import HTML

In [ ]:
HTML(response['choices'][0]['message']['content'])

Store Name,Date,Item Name,Item Category,Price
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,CHARCOAL 8KG,Home improvements and constructions,17.99
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,BAMBOO DRAWR,Home improvements and constructions,82.99
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,RUFFLES ALL,Food items,5.99
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,CK BRA 2PK,Beauty and healthcare,19.99
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,DENSIFY,Home improvements and constructions,19.99
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,TPD/1632921,Home improvements and constructions,4.00-
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,ORG BEANS,Food items,1.99
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,RIVAJ NAAN,Food items,1.99
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,MUSHROOMS,Food items,4.39
COSTCO WHOLESALE Kanata #541,2022/06/03 20:02:00,TPD/1596,Food items,5.49
